In [32]:
!pip install tensorflow keras numpy matplotlib pandas sklearn biopython

3Requirement already satisfied: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.0)


# **1. Initialize Network**

In [33]:
from Bio import SeqIO
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from random import seed
from random import random
from random import randrange
from csv import reader
from math import exp
import re, ast


# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


Running the example, you can see that the code prints out each layer one by one. You can see the hidden layer has one neuron with 2 input weights plus the bias. The output layer has 2 neurons, each with 1 weight plus the bias.

# **2. Forward Propagate**
We can break forward propagation down into three parts:

* Neuron Activation.
* Neuron Transfer.
* Forward Propagation.

### **2.1. Neuron Activation**
**activation = sum(weight_i * input_i) + bias**

In [34]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


### **2.2. Neuron Transfer**
Once a neuron is activated, we need to transfer the activation to see what the neuron output actually is.

Different transfer functions can be used. It is traditional to use the sigmoid activation function, but you can also use the tanh (hyperbolic tangent) function to transfer outputs. More recently, the rectifier transfer function has been popular with large deep learning networks.

The sigmoid activation function looks like an S shape, it’s also called the logistic function. It can take any input value and produce a number between 0 and 1 on an S-curve. It is also a function of which we can easily calculate the derivative (slope) that we will need later when backpropagating error.

We can transfer an activation function using the sigmoid function as follows:

### **output = 1 / (1 + e^(-activation))**

In [35]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

### **2.3. Forward Propagation**
Forward propagating an input is straightforward.

We work through each layer of our network calculating the outputs for each neuron. All of the outputs from one layer become inputs to the neurons on the next layer.

Below is a function named forward_propagate() that implements the forward propagation for a row of data from our dataset with our neural network.

You can see that a neuron’s output value is stored in the neuron with the name ‘output‘. You can also see that we collect the outputs for a layer in an array named new_inputs that becomes the array inputs and is used as inputs for the following layer.

The function returns the outputs from the last layer also called the output layer.

In [36]:

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# **3. Back Propagate Error**

The backpropagation algorithm is named for the way in which weights are trained.
Error is calculated between the expected outputs and the outputs forward propagated from the network. These errors are then propagated backward through the network from the output layer to the hidden layer, assigning blame for the error and updating weights as they go.

The math for backpropagating error is rooted in calculus, but we will remain high level in this section and focus on what is calculated and how rather than why the calculations take this particular form.

This part is broken down into two sections.

* Transfer Derivative.
* Error Backpropagation.

### **3.1. Transfer Derivative**
Given an output value from a neuron, we need to calculate it’s slope.

We are using the sigmoid transfer function, the derivative of which can be calculated as follows:

**derivative = output * (1.0 - output)**
Below is a function named transfer_derivative() that implements this equation.

In [37]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

### **3.2. Error Backpropagation**
The first step is to calculate the error for each output neuron, this will give us our error signal (input) to propagate backwards through the network.

The error for a given neuron can be calculated as follows:

**error = (expected - output) * transfer_derivative(output)**

The error signal for a neuron in the hidden layer is calculated as the weighted error of each neuron in the output layer. Think of the error traveling back along the weights of the output layer to the neurons in the hidden layer.

**error = (weight_k * error_j) * transfer_derivative(output)**

Where error_j is the error signal from the jth neuron in the output layer, weight_k is the weight that connects the kth neuron to the current neuron and output is the output for the current neuron.

Below is a function named backward_propagate_error() that implements this procedure.

In [38]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

You can see that the error signal calculated for each neuron is stored with the name ‘delta’. You can see that the layers of the network are iterated in reverse order, starting at the output and working backwards. This ensures that the neurons in the output layer have ‘delta’ values calculated first that neurons in the hidden layer can use in the subsequent iteration. I chose the name ‘delta’ to reflect the change the error implies on the neuron (e.g. the weight delta).

You can see that the error signal for neurons in the hidden layer is accumulated from neurons in the output layer where the hidden neuron number j is also the index of the neuron’s weight in the output layer neuron[‘weights’][j].

In [39]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


# **4. Train Network**
The network is trained using stochastic gradient descent.

This involves multiple iterations of exposing a training dataset to the network and for each row of data forward propagating the inputs, backpropagating the error and updating the network weights.

This part is broken down into two sections:

* Update Weights.
* Train Network.

### **4.1. Update Weights**

Once errors are calculated for each neuron in the network via the back propagation method above, they can be used to update weights.

Network weights are updated as follows:

**weight = weight + learning_rate * error * input**

Where weight is a given weight, learning_rate is a parameter that you must specify, error is the error calculated by the backpropagation procedure for the neuron and input is the input value that caused the error.

In [40]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

### **4.2. Train Network**

In [41]:

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


Below is the complete example. We will use 2 neurons in the hidden layer. It is a binary classification problem (2 classes) so there will be two neurons in the output layer. The network will be trained for 20 epochs with a learning rate of 0.5, which is high because we are training for so few iterations.

# **5. Predict**

In [42]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [43]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [44]:
# Python program to convert decimal to binary 
    
# Function to convert Decimal number  
# to Binary number  
def decimalToBinary(n,n_bit):  
    p="0"+n_bit+"b"
    res = str([int(i) for i in list(format(n,p))] )
    return res  

In [45]:
# Python3 program to convert  
# list into a list of lists 
  
def extractDigits(lst): 
    return [[el] for el in lst] 

In [61]:

# Driver code  
if __name__ == '__main__': 

  id=[]
  bit = []
  target=[]
  dataset=[]
  data=[]
  #input the number of bits
  n_bit=input("enter :")
  #calculate the total no of samples
  n_total= 2** int(n_bit)
  #taking the half of the tatal
  n_half=int(n_total/2)
  #decimal to binary
  for i in range(n_total):
    bin = decimalToBinary(i,n_bit)
    bit.append(bin)
    id.append(i)
  #creating target(1) for first half
  for j in range(0,n_half):
    target.append(1)

  #creating target(0) for second half
  for k in range(n_half,n_total):
    target.append(0)

  #creating Dataframe
  raw_data ={
              'ID':id,
              'Bit': bit,
              'Target':target
              }
  df=pd.DataFrame(raw_data,columns=['ID','Bit','Target'])
  is_even = df['ID'].astype(int) % 2 == 0

  df_train = df[is_even]
  df_test = df[~is_even]

  print("Training Dataset :\n",df_train)
  print("Testing Dataset :\n",df_test)
  

  #loading train data
  X=df_train.Bit.values.tolist()
  Y=df_train.Target.values.tolist()
  #convert the list of strings to list of list
  data = [list(map(int,k.strip("[]").split(", "))) for k in X]
  #convertthe list of integer to list of list
  t= extractDigits(Y)
  #merger elementwise the data and corresponding target
  dataset= [i + j for i, j in zip(data, t)] 
  print("TrainSet :\n",dataset)

  seed(1)
  #loading test data
  X1=df_test.Bit.values.tolist()
  Y1=df_test.Target.values.tolist()
  y_test=Y1
  #convert the list of strings to list of list
  test_data = [list(map(int,k.strip("[]").split(", "))) for k in X1]
  #convertthe list of integer to list of list
  test_t= extractDigits(Y1)
  #merger elementwise the data and corresponding target
  testset= [i + j for i, j in zip(test_data, test_t)] 
  print("TestSet :\n",testset)

  y_pred=back_propagation(dataset, testset, l_rate=0.005, n_epoch=150, n_hidden=5)
  print(y_test)
  print(y_pred)
  

enter :6
Training Dataset :
     ID                 Bit  Target
0    0  [0, 0, 0, 0, 0, 0]       1
2    2  [0, 0, 0, 0, 1, 0]       1
4    4  [0, 0, 0, 1, 0, 0]       1
6    6  [0, 0, 0, 1, 1, 0]       1
8    8  [0, 0, 1, 0, 0, 0]       1
10  10  [0, 0, 1, 0, 1, 0]       1
12  12  [0, 0, 1, 1, 0, 0]       1
14  14  [0, 0, 1, 1, 1, 0]       1
16  16  [0, 1, 0, 0, 0, 0]       1
18  18  [0, 1, 0, 0, 1, 0]       1
20  20  [0, 1, 0, 1, 0, 0]       1
22  22  [0, 1, 0, 1, 1, 0]       1
24  24  [0, 1, 1, 0, 0, 0]       1
26  26  [0, 1, 1, 0, 1, 0]       1
28  28  [0, 1, 1, 1, 0, 0]       1
30  30  [0, 1, 1, 1, 1, 0]       1
32  32  [1, 0, 0, 0, 0, 0]       0
34  34  [1, 0, 0, 0, 1, 0]       0
36  36  [1, 0, 0, 1, 0, 0]       0
38  38  [1, 0, 0, 1, 1, 0]       0
40  40  [1, 0, 1, 0, 0, 0]       0
42  42  [1, 0, 1, 0, 1, 0]       0
44  44  [1, 0, 1, 1, 0, 0]       0
46  46  [1, 0, 1, 1, 1, 0]       0
48  48  [1, 1, 0, 0, 0, 0]       0
50  50  [1, 1, 0, 0, 1, 0]       0
52  52  [1, 1, 0, 1, 0, 0]


# **Evaluating Performance**

In [62]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(y_test,y_pred) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(y_test,y_pred))
print ('Report : ')
print (classification_report(y_test,y_pred))

Confusion Matrix :
[[16  0]
 [12  4]]
Accuracy Score : 0.625
Report : 
              precision    recall  f1-score   support

           0       0.57      1.00      0.73        16
           1       1.00      0.25      0.40        16

    accuracy                           0.62        32
   macro avg       0.79      0.62      0.56        32
weighted avg       0.79      0.62      0.56        32



# **Reference:**
1. [https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/)
